In [149]:
import pandas as pd
from Bio import SeqIO, AlignIO, Seq
import numpy as np
from collections import Counter
from helper import *
import python_cipres.client as CipresClient

### Extract fasta sequences for the autotrophic rubiscos
Before running this code we need to generate a file called `autotrophic_rubisco_70p.csv` by selecting in ITOL clades that belong to Rubisco types 1,2,2/3,3a,3c,3-like,3b and IV. We replaced sapces in the sequence ID to `_` to match the sequence ID in the fasta files. We take only sequences which are not type IV or type III-b as autotrophic.

In [50]:
uclust_data = pd.read_csv('../output/01_70p_tree/uclust_all_0.7.csv')
uclust_data['cut Target'] = uclust_data.Target.apply(lambda x: x.split(' ')[0])
#auto_id = [x.replace('\n','') for x in open('../output/01_70p_tree/autotrophic_rubisco_70p.txt').readlines() if x != '\n']
rubisco_types = pd.read_csv('../output/01_70p_tree/rubisco_types_70p.csv')

rubisco_with_type = uclust_data.merge(rubisco_types, left_on='cut Target', right_on='ID')
rubisco_with_type[rubisco_with_type.type != 'IV'].groupby('type')['Query'].count()

type
I         32335
II          338
II/III       77
III         815
Name: Query, dtype: int64

In [27]:
len(uclust_data)

35413

In [6]:
uclust_data = pd.read_csv('../output/01_70p_tree/uclust_all_0.7.csv')
uclust_data['cut Target'] = uclust_data.Target.apply(lambda x: x.split(' ')[0])
#auto_id = [x.replace('\n','') for x in open('../output/01_70p_tree/autotrophic_rubisco_70p.txt').readlines() if x != '\n']
rubisco_types = pd.read_csv('../output/01_70p_tree/rubisco_types_70p.csv')
carbo_rucisco = rubisco_types[~rubisco_types['type'].isin(['IV'])]
auto_rubisco = uclust_data.merge(carbo_rucisco, left_on='cut Target', right_on='ID')
true_rubisco  = auto_rubisco['Query'].values

auto_seq = []
for record in SeqIO.parse('../output/00_100p_tree/uclust_all_1.faa', "fasta"):
    if record.description in true_rubisco:
        auto_seq.append(record)
!mkdir -p ../output/02_90p_autotrophic_rubisco_tree
with open(r"../output/02_90p_autotrophic_rubisco_tree/true_rubisco_seq.faa", "w") as output_handle:
    SeqIO.write(auto_seq, output_handle, "fasta")

### Cluster sequences using uclust

In [7]:
!../bin/usearch11.0.667_i86linux32 -cluster_fast ../output/02_90p_autotrophic_rubisco_tree/true_rubisco_seq.faa -id 0.9 -uc ../output/02_90p_autotrophic_rubisco_tree/true_uclust_all_0.9.uc

usearch v11.0.667_i86linux32, 4.0Gb RAM (16.1Gb total), 4 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: yinonmoise.baron@weizmann.ac.il

00:00 62Mb    100.0% Reading ../output/02_90p_autotrophic_rubisco_tree/true_rubisco_seq.faa
00:00 59Mb    100.0% DF
00:00 60Mb   33565 seqs, 33565 uniques, 33565 singletons (100.0%)
00:00 60Mb   Min size 1, median 1, max 1, avg 1.00
00:00 67Mb    100.0% DB
00:03 129Mb   100.0% 1167 clusters, max size 6202, avg 28.8
                                                           
      Seqs  33565 (33.6k)
  Clusters  1167
  Max size  6202
  Avg size  28.8
  Min size  1
Singletons  672, 2.0% of seqs, 57.6% of clusters
   Max mem  129Mb
      Time  3.00s
Throughput  11.2k seqs/sec.



### Take only cluster fasta files and create csv file

In [8]:
parse_uclust(infile='../output/02_90p_autotrophic_rubisco_tree/true_uclust_all_0.9.uc',
             fasta='../output/02_90p_autotrophic_rubisco_tree/true_rubisco_seq.faa',
             outfasta='../output/02_90p_autotrophic_rubisco_tree/true_uclust_all_0.9.faa',
             outfile='../output/02_90p_autotrophic_rubisco_tree/true_uclust_all_0.9.csv'
            )

In [31]:
true_rubisco = pd.read_csv('../output/02_90p_autotrophic_rubisco_tree/true_uclust_all_0.9.csv')
jaffe = pd.read_csv('../data/jaffe_et_al_2018_rubisco_types_processed.csv')
true_rubisco = true_rubisco[~true_rubisco.Query.isin(jaffe.ID)]
true_rubisco['ID'] = true_rubisco.Target.apply(lambda x: x.split(' ')[0])

#true_rubisco_with_type = true_rubisco.merge(rubisco_types, left_on='ID', right_on='ID')
true_rubisco_with_type = uclust_data.merge(true_rubisco,left_on='Query',right_on='Query',suffixes=('_70','_90'))
true_rubisco_with_type = true_rubisco_with_type.merge(rubisco_types,left_on='cut Target', right_on='ID')
true_rubisco_with_type.groupby('type')['Query'].count()
true_rubisco_with_type.groupby('type')['Target_90'].nunique()

type
I         460
II        119
II/III     23
III       231
Name: Target_90, dtype: int64

In [37]:
auto_rubisco_with_type = true_rubisco_with_type[true_rubisco_with_type.type.isin(['I','II','II/III'])]
auto_rubisco_with_type.to_csv('../output/02_90p_autotrophic_rubisco_tree/auto_rubisco_0.9_with_type.csv')
auto_rubisco = true_rubisco[true_rubisco.Query.isin(auto_rubisco_with_type.Query)]
auto_rubisco.to_csv('../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.csv')
auto_rubisco_ID  = auto_rubisco['Target'].unique()

auto_seq = []
for record in SeqIO.parse('../output/02_90p_autotrophic_rubisco_tree/true_uclust_all_0.9.faa', "fasta"):
    if record.description in auto_rubisco_ID:
        auto_seq.append(record)
!mkdir -p ../output/02_90p_autotrophic_rubisco_tree
with open(r"../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.faa", "w") as output_handle:
    SeqIO.write(auto_seq, output_handle, "fasta")

### Remove outliers - sequences which less than 50% of them map to Rr sequence by blast

In [38]:
from Bio.Blast.Applications import NcbiblastpCommandline
output = NcbiblastpCommandline(query="../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.faa", subject="../data/Rr.faa", outfmt=5)()[0]
from Bio.Blast import NCBIXML
from io import StringIO

res = []
for x in NCBIXML.parse(StringIO(output)):
    if len(x.alignments) == 0:
        res.append(0)
    else:
        alignment = x.alignments[0]
        res.append(pd.DataFrame([[x.align_length,x.expect] for x in alignment.hsps]).sort_values(by=1).loc[0,0]/x.query_length)

seq = [x for x in SeqIO.parse('../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.faa',format='fasta')]

res_df = pd.Series(res,index=[x.id for x in seq])

In [39]:
seqs = [x for x in seq if x.id in res_df[res_df >= 0.5].index]

with open(r"../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_no_outliers.faa", "w") as output_handle:
    SeqIO.write(seqs, output_handle, "fasta")
    
seqs = [x for x in seq if x.id in res_df[res_df < 0.5].index]
with open(r"../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_outliers.faa", "w") as output_handle:
    SeqIO.write(seqs, output_handle, "fasta")

## Create multiple sequence alignment

In [42]:
#!../bin/mafft-linux64/mafft.bat ../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.faa > ../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.aln
!../bin/mafft-linux64/mafft.bat ../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_no_outliers.faa > ../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.aln

nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
  501 / 599
done.

Constructing a UPGMA tree (efffree=0) ... 
  590 / 599
done.

Progressive alignment 1/2... 
STEP   393 / 598 
Reallocating..done. *alloclen = 2374
STEP   501 / 598 
done.

Making a distance matrix from msa.. 
  500 / 599
done.

Constructing a UPGMA tree (efffree=1) ... 
  590 / 599
done.

Progressive alignment 2/2... 
STEP   501 / 598 
Reallocating..done. *alloclen = 2368

done.

disttbfast (aa) Version 7.427
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gap

## Clean MSA to contain only positions with more than 5% coverage (based on Jaffe et al. 2018)

In [43]:
clean_aln(infile='../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.aln',
          outfile='../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_trimmed.aln')

In [44]:
cip = CipresClient.Client(appname='RO',
                    appID='rubisco_phylogeny-49F87B124F3D429FBE12F95E4254DDEA',
                    baseUrl='https://cipresrest.sdsc.edu/cipresrest/v1',
                    username='yinonbaron',
                    password='Mchcav11~')


In [45]:
job = cip.submitJob(vParams={'toolId': 'RAXMLHPC8_REST_XSEDE',
                       'datatype_': 'protein',
                       'runtime_': '160',
                       'select_analysis_': 'fa',
                       'choose_bootstrap_': 'x',
                       'printbrlength_': '1'},
              inputParams={'infile_': '../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_trimmed.aln'},
              metadata={'statusEmail':'true'})

In [10]:
job = cip.listJobs()[-1]

In [49]:
if cip.getJobStatus(jobHandle=job.jobHandle).isDone():
    !mkdir -p ../output/02_90p_autotrophic_rubisco_tree/RaxML/
    job.downloadResults('../output/02_90p_autotrophic_rubisco_tree/RaxML/')
else:
    print('Job ' + job.jobHandle + ' not finished')

In [52]:
auto_rubisco[auto_rubisco.Query=='gi|363498367|gb|AAQ04822.2|AF463409_1']

,Type,Cluster,Size,%Id,Strand,Qlo,Tlo,Alignment,Query,Target,ID


In [6]:
d = pd.read_csv('../output/00_100p_tree/uclust_all_1_rubisco_types.csv')
d2 = pd.read_csv('../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_no_outliers.csv')
d3 = d2.merge(d, left_on='Query', right_on='ID',how='left')
d3.type.unique()

array([nan, 'I', 'IIIlike', 'II', 'II/III', 'IIIc', 'unknown', 'IIIa',
       'IIIb'], dtype=object)

In [55]:
add_type(type_file='../output/00_100p_tree/uclust_all_1_rubisco_types.csv',
         seq_file='../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_no_outliers.csv',
         outfile='../output/02_90p_autotrophic_rubisco_tree/type_legend.txt')

In [150]:
add_kinetic(kinetic_file='../output/00_100p_tree/uclust_all_1_kinetic_data.csv',
         synth_file='../output/00_100p_tree/synth_data_modified.csv',
         seq_file='../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_no_outliers.csv',
         outfile='../output/02_90p_autotrophic_rubisco_tree/kinetic_legend.txt')

In [53]:
add_kinetic_on_label(kinetic_file='../output/00_100p_tree/uclust_all_1_kinetic_data.csv',
         synth_file='../output/00_100p_tree/synth_data.csv',
         seq_file='../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_no_outliers.csv',
         outfile='../output/02_90p_autotrophic_rubisco_tree/kinetic_legend_label.txt')


['RBC2_44,label,node,#B2BABB,1,normal\n'
 'RBC2_44,label,node,#B2BABB,1,normal\n'
 'RBC2_44,label,node,#B2BABB,1,normal\n' ...
 'gi|357471525|ref|XP_003606047.1|,label,node,#B2BABB,1,normal\n'
 'gi|387767798|gb|AFJ96478.1|,label,node,#B2BABB,1,normal\n'
 'RBC2_50,label,node,#B2BABB,1,normal\n']


In [23]:
k = pd.read_csv('../output/00_100p_tree/uclust_all_1_kinetic_data.csv')
k

,kinetic_ID
0,RBC2_50 gi|499819577|ref|WP_011500311.1|
1,gi|499561661|ref|WP_011242444.1| MULTISPECIES:...
2,gi|499709017|ref|WP_011389751.1| ribulose-bisp...
3,gi|502736740|ref|WP_012971724.1| ribulose bisp...
4,"gi|356472757|gb|AET10441.1| ribulose-1,5-bisph..."
5,"gi|11467200|ref|NP_043033.1| ribulose-1,5-bisp..."
6,"gi|14017580|ref|NP_114267.1| ribulose-1,5-bisp..."
7,"gi|11497536|ref|NP_054944.1| ribulose-1,5-bisp..."
8,"gi|1248646408|gb|ATG33856.1| ribulose-1,5-bisp..."
9,gi|157678845|dbj|BAF80663.1| ribulose 1.5-bisp...


In [1]:
uclust = rubisco_types = pd.read_csv('../output/01_70p_tree/rubisco_types_70p.csv')
uclust70 = pd.read_csv('../output/01_70p_tree/uclust_all_0.7.csv')
rubisco_types = pd.read_csv('../output/01_70p_tree/rubisco_types_70p.csv')
#uclust[uclust.Query.str.contains('499819577')]
#uclust70[uclust70.Query.str.contains('499819577')]


In [109]:
uclust = pd.read_csv('../output/02_90p_autotrophic_rubisco_tree/auto_rubisco_0.9_with_type.csv')
t = uclust[uclust.type.isin(['II','II/III'])]
#syn = pd.read_csv('../output/00_100p_tree/synth_data.csv',header=None)
#syn = pd.read_csv('../output/00_100p_tree/synth_data_modified.csv',header=None)
syn = pd.read_csv('../output/00_100p_tree/milo_syn_100p.csv')
#mdata = t.merge(syn,left_on='Query',right_on=0,how='outer')
mdata = t.merge(syn,left_on='Query',right_on='Target',how='outer')
#mdata[~mdata.Target_90.isin(mdata[~mdata[0].isna()].Target_90.unique())].Target_90.unique()
missing_IDs = mdata[mdata.Query.isin(mdata[~mdata.Target_90.isin(mdata[~mdata['0'].isna()].Target_90.unique())].Target_90.unique())]

,Unnamed: 0_x,Type_70,Cluster_70,Size_70,%Id_70,Strand_70,Qlo_70,Tlo_70,Alignment_70,Query,...,Tlo_90,Alignment_90,Target_90,ID_x,ID_y,type,Unnamed: 0_y,0,Target,Internal ID
121,118.0,H,0.0,459.0,80.3,.,0,0,457M2D,RBG_16_Gammaproteobacteria_62_13_RBG_16_scaffo...,...,*,*,RBG_16_Gammaproteobacteria_62_13_RBG_16_scaffo...,RBG_16_Gammaproteobacteria_62_13_RBG_16_scaffo...,RBC_81,II,NaN,NaN,NaN,NaN
125,122.0,H,0.0,459.0,79.9,.,0,0,457M2D,GWA2_Gallionellales_59_43_gwa2_scaffold_2131_2...,...,*,*,GWA2_Gallionellales_59_43_gwa2_scaffold_2131_2...,GWA2_Gallionellales_59_43_gwa2_scaffold_2131_28,RBC_81,II,NaN,NaN,NaN,NaN
134,131.0,H,0.0,459.0,79.6,.,0,0,457M2D,"gi|1232577783|gb|OYY18796.1| ribulose 1,5-bisp...",...,*,*,"gi|1232577783|gb|OYY18796.1| ribulose 1,5-bisp...",gi|1232577783|gb|OYY18796.1|,RBC_81,II,NaN,NaN,NaN,NaN
146,143.0,H,0.0,386.0,85.2,.,0,0,386M71I,"gi|1176749570|gb|OQY67889.1| ribulose 1,5-bisp...",...,*,*,"gi|1176749570|gb|OQY67889.1| ribulose 1,5-bisp...",gi|1176749570|gb|OQY67889.1|,RBC_81,II,NaN,NaN,NaN,NaN
178,175.0,H,0.0,459.0,80.1,.,0,0,457M2D,"gi|1232622198|gb|OYY58774.1| ribulose 1,5-bisp...",...,*,*,"gi|1232622198|gb|OYY58774.1| ribulose 1,5-bisp...",gi|1232622198|gb|OYY58774.1|,RBC_81,II,NaN,NaN,NaN,NaN
361,1209.0,C,35.0,11.0,*,*,*,*,*,"gi|37730371|gb|AAO13083.1| ribulose 1,5-bispho...",...,*,*,"gi|37730371|gb|AAO13083.1| ribulose 1,5-bispho...",gi|37730371|gb|AAO13083.1|,gi|37730371|gb|AAO13083.1|,II,NaN,NaN,NaN,NaN


In [144]:
old_clust = pd.read_csv('../../../../../RO/UCLUST 2017/usearch_results.0.5-1.0/clusters/type_II.faa.sorted.0.90.uc',sep='\t',header=None)
old_clust = old_clust[old_clust[0] !='S']
old_clust.loc[old_clust[9] =='*',9] =old_clust.loc[old_clust[9] =='*',8]
old_clust[8] = old_clust[8].str.replace('__','_')
old_clust[9]=old_clust[9].str.replace('RBC','RBC_')
#old_clust[8].replace('__','_',inplace=True)
o = missing_IDs.merge(old_clust,left_on='ID_x',right_on=8)
lib = pd.read_csv('../../data/milo_variant_data.csv')
e = lib[lib['Identifier'].isin(o[9])]
e2 = lib[lib['Internal ID'].isin(o[9])]
missing_in_lib = pd.concat([e,e2])

k = pd.read_csv('../output/00_100p_tree/uclust_all_1.csv')
k['QID'] = k['Query'].apply(lambda x: x.split(' ')[0])
k2 = k[k.QID.isin(missing_in_lib['Identifier'])]
o
#syn[syn['0'].isin(k2.Target)]

#k2
mdata[~mdata['0'].isna()].groupby('type').Target_90.nunique()

type
II        113
II/III     23
Name: Target_90, dtype: int64

In [147]:
syn['']

,Unnamed: 0,0,Target,Internal ID
0,0,RBC3_2 WP_048136654.1,RBC2_49 gi|851262160|ref|WP_048136654.1|,RBC3_2
1,1,RBC_67 gi|499792604|ref|WP_011473338.1|,RBC_66 gi|499792604|ref|WP_011473338.1|,RBC_67
2,2,RBC4_78 RIFOXYB2_FULL_PER_41_88_rifoxyb2_full_...,RBC2_4 gi|406927721|gb|EKD63705.1|,RBC4_78
3,3,Rr gi|56565978|gb|AAV98336.1|,gi|56565978|gb|AAV98336.1| synthetic ribulose-...,Rr
4,4,RBC2_39 gi|502832577|ref|WP_013067553.1|,RBC4_77 gi|954037512|gb|ALP32073.1|,RBC2_39
5,5,RBC2_50 gi|499819577|ref|WP_011500311.1|,RBC2_50 gi|499819577|ref|WP_011500311.1|,RBC2_50
6,6,RBC_96[G359W] gi|655952301|ref|WP_028995230.1|,RBC_96 gi|655952301|ref|WP_028995230.1|,RBC_96[G359W]
7,7,RBC_89 gi|491010518|ref|WP_004872227.1|,RBC_89 gi|491010518|ref|WP_004872227.1|,RBC_89
8,8,RBC_123 gi|501530536|ref|WP_012537012.1|,RBC_123 gi|501530536|ref|WP_012537012.1|,RBC_123
9,9,AfM tr|B7J5E3|B7J5E3_ACIF2,RBC_123 gi|501530536|ref|WP_012537012.1|,AfM
